In [1]:
# In this example we will show the difference between fixes computed with laika
# from raw data of the ublox receiver vs the the fixes the ublox receiver
# computes

In [14]:
import numpy as np

base_dir = 'example_data'


with open(f'{base_dir}/raw_gnss_ublox/t', 'rb') as f:
  raw_ublox_t = np.load(f)
with open(f'{base_dir}/raw_gnss_ublox/value', 'rb') as f:
  raw_ublox = np.load(f)
with open(f'{base_dir}/live_gnss_ublox/t', 'rb') as f:
  fixes_ublox_t = np.load(f)
with open(f'{base_dir}/live_gnss_ublox/value', 'rb') as f:
  fixes_ublox = np.load(f)

In [15]:
# We get the raw data into our format from the log array format

from laika.raw_gnss import normal_meas_from_array
measurements = np.array([normal_meas_from_array(arr) for arr in raw_ublox])

In [16]:
# initialize an astrodog with dgps corrections

from laika import AstroDog
dog = AstroDog(dgps=True)

In [17]:
# Building this cache takes forever just copy it from repo

from shutil import copyfile
import os
cache_directory = '/tmp/gnss/cors_coord/'
try:
  os.mkdir('/tmp/gnss/')
except:
  pass
try:
  os.mkdir(cache_directory)
except:
  pass
copyfile('cors_station_positions', cache_directory + 'cors_station_positions')

'/tmp/gnss/cors_coord/cors_station_positions'

In [18]:
from laika.raw_gnss import process_measurements, correct_measurements, calc_pos_fix
from tqdm.auto import tqdm

# We want to group measurements by measurement epoch
# this makes the kalman filter faster and is easier
# to reason about
grouped_t = sorted(list(set(raw_ublox_t)))                                                                                      
grouped_meas_processed = []
corrected_meas_arrays = []

# process measurement groups
for t in grouped_t:
  meas = measurements[raw_ublox_t == t]
  grouped_meas_processed.append(process_measurements(meas, dog))

# correct measurement groups with an estimate position
# that was computes with weighted-least-squares on
# the first epoch
# WARNING: can take up to 10min
wls_estimate = calc_pos_fix(grouped_meas_processed[0])
est_pos = wls_estimate[0][:3]
for proc in tqdm(grouped_meas_processed):
  corrected = correct_measurements(proc, est_pos, dog)
  corrected_meas_arrays.append(np.array([c.as_array() for c in corrected]))


  0%| | 0/600 [00:00<?, ?it/s]

HTTPS error 404
HTTPS error 404
File not downloaded, check availability on server.
HTTPS error 404
HTTPS error 404
File not downloaded, check availability on server.
HTTPS error 404
HTTPS error 404
File not downloaded, check availability on server.


100%|█| 600/600 [01:06<00:00,  8.97it/s]  


In [19]:
for proc in tqdm(grouped_meas_processed):
  corrected = correct_measurements(proc, est_pos, dog)
  corrected_meas_arrays.append(np.array([c.as_array() for c in corrected]))

100%|█| 600/600 [00:02<00:00, 205.30it/s]


In [20]:
# We run the kalman filter

from kalman.models.gnss_kf import GNSSKalman
from kalman.kalman_helpers import run_car_ekf_offline, ObservationKind
ekf = GNSSKalman()
init_state = ekf.x
init_state[:3] = est_pos
ekf.init_state(init_state)
ekf_data = {}
ekf_data[ObservationKind.PSEUDORANGE_GPS] = (grouped_t, corrected_meas_arrays)
ekf_data[ObservationKind.PSEUDORANGE_RATE_GPS] = (grouped_t, corrected_meas_arrays)
ekf_outputs = run_car_ekf_offline(ekf, ekf_data)

import laika.lib.coordinates as coord
laika_positions_t = ekf_outputs[4]
laika_positions_ecef = ekf_outputs[0][:,:3]
laika_positions_geodetic = coord.ecef2geodetic(laika_positions_ecef)

100%|█| 1200/1200 [00:03<00:00, 316.10it/s]
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3444: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
ublox_positions_geodetic = fixes_ublox[:,[0,1,4]]

In [22]:
# By looking at the map, we can see that the two paths compared.
# If you want to regenerate the gmplot you will need a google
# maps API key

import gmplot
gmap = gmplot.GoogleMapPlotter(*laika_positions_geodetic[0])
#gmap.apikey='...'
gmap.plot([x[0]  for x in laika_positions_geodetic], [x[1] for x in laika_positions_geodetic], 'blue', edge_width = 5)
gmap.plot([x[0]  for x in ublox_positions_geodetic], [x[1] for x in ublox_positions_geodetic], 'red', edge_width = 5)
gmap.draw("laika_quality_check.html")



import webbrowser
import os
webbrowser.open('file://' + os.path.realpath("laika_quality_check.html"));